In [2]:
import numpy as np
import importlib
import matplotlib.pyplot as plt
import pickle
from IPython.display import clear_output
# import bokeh.io
# import bokeh.plotting
# import bokeh.models

In [ ]:
import qubelsi.qube
import inspect

In [ ]:
qube = qubelsi.qube.Qube('10.5.0.23',"/home/qube/adi_api_mod") # Riken1-05
#qube.do_init(bitfile="/home/qube/bin/069414.bit", message_out=True)
#qube.do_init()

In [1]:
import pyvisa
import time
m = pyvisa.ResourceManager('@py')
inst = m.open_resource("TCPIP0::10.250.0.10::inst0::INSTR")
inst.query('*IDN?')

'"Anritsu,MS2038C/19,2146021,4.30"'

In [3]:
# MS2038Cから測定データを読み出す
def get_freq_from_ms2038c(inst):
    myfind = lambda k, l: next(filter(lambda x: x.startswith("{}=".format(k)), l))
    myconv = lambda s: float(s.split('=')[1].replace(' MHZ', ''))
    inst.timeout = 10000
    preamble = inst.query(':TRAC:PRE?')[10:].split(',')
    fstart = myconv(myfind('START_FREQ', preamble))
    fstop = myconv(myfind('STOP_FREQ', preamble))
    npts = int(float(myfind('UI_DATA_POINTS', preamble).split('=')[1]))
    return np.linspace(fstart, fstop, npts)
    
def get_trace_from_ms2038c(inst):
    return np.loadtxt(inst.query(':TRAC?')[6:].split(','))

In [ ]:
offset = lambda l, d: [v + i*d for i, v in enumerate(l)]

def plot_spurious_multi(p, d):
    p.renderers = []
    p.multi_line(offset(d['freq'],10), offset(d['trace'],0))
    bokeh.plotting.show(p)    
    
def plot_spurious(p, d, i):
    p.renderers = []
    p.line(d['freq'][i], d['trace'][i])
    bokeh.plotting.show(p)    

In [ ]:
import examples.long_send.long_send
import examples.long_send.stop_awgs

In [ ]:
from collections import namedtuple

Vatt = namedtuple('Vatt', ('dac', 'ch'))
Upconv = namedtuple('Upconv', ('vatt',))
Ifdac = namedtuple('Ifdac', ('ad9082', 'ch'))
CtrlPort = namedtuple('CtrlPort', ('losc', 'ifdac', 'upconv'))

port = {
    5 : CtrlPort(qube.lmx2594[2], Ifdac(qube.ad9082[0], 2), Upconv(Vatt(qube.ad5328, 2))), # CTRL1
    6 : CtrlPort(qube.lmx2594[3], Ifdac(qube.ad9082[0], 3), Upconv(Vatt(qube.ad5328, 3))), # CTRL2
    7 : CtrlPort(qube.lmx2594[4], Ifdac(qube.ad9082[1], 0), Upconv(Vatt(qube.ad5328, 4))), # CTRL3
    8 : CtrlPort(qube.lmx2594[5], Ifdac(qube.ad9082[1], 1), Upconv(Vatt(qube.ad5328, 5))), # CTRL4
}

def apply_lmx2594(o):
    o.write_value(0x00, 0x6418)
    return True
    
def set_lmx2594_OUTA_PD(o, b):
    if b:
        v = o.read_value(44) & 0b1111111110111111
    else:
        v = o.read_value(44) | 0b0000000001000000
    o.write_value(44, v)
    return v

def set_lmx2594_OUTB_PD(o, b):
    if b:
        v = o.read_value(44) & 0b1111111101111111
    else:
        v = o.read_value(44) | 0b0000000010000000
    o.write_value(44, v)
    return v

def set_lmx2594_OUTA_PWR(o, n): # 0 - 63
    v = o.read_value(44) & 0b1100000011111111 | n * 0x100
    o.write_value(44, v)
    return v

def set_lmx2594_OUTB_PWR(o, n): # 0 - 63
    v = o.read_value(45) & 0b1111111111000000 | n
    o.write_value(45, v)
    return v

def set_lmx2594_freq_100M(lmx2594, n):
    lmx2594.write_value(0x24, n)
    return n

def set_flo_according_to_frf(lmx2594, frf, fif, apply=True): # MHz
    flo = frf + fif
    set_lmx2594_freq_100M(lmx2594, int(flo*1e-2))
    if apply:
        apply_lmx2594(lmx2594)
    return flo

def set_flo_fnco_according_to_frf(port, frf, fif, apply=True): #MHz
    fif = 1900
    fnco = fif - 2.5
    ifdac = port.ifdac
    ifdac.ad9082.set_nco(ch=ifdac.ch, freq=fnco*1e+6)
    flo = set_flo_according_to_frf(port.losc, frf, fif, apply=apply)
    return flo, fnco
    
def read_peak(inst, fc): # MHz
    #inst.write(':INIT:CONT OFF; *WAI')
    inst.write(':FREQ:CENT {} MHZ; *WAI'.format(fc))
    #inst.write(':INIT ONCE')
    time.sleep(2)
    inst.write('CALC:MARK1:MAX')
    x = inst.query('CALC:MARK1:X?')
    y = inst.query('CALC:MARK1:Y?')
    #inst.write(':INIT:CONT ON; *WAI')
    return x, y

def apply_vatt(ad5328):
    ad5328.write_value(0xA, 0x002)
    return True

def set_vatt(vatt, v, apply=True): # max 4095
    vatt.dac.write_value(vatt.ch, v)
    if apply:
        apply_vatt(vatt.dac)
    return v/0xfff*3.3

#read_peak(inst, 8500)

In [ ]:
# ADRF6780のVATTを変更しながらスペクトルを取得する
set_flo_fnco_according_to_frf(port[8], 9000, 3000)

def sweep_ms2038c(inst):
    inst.write(':INIT')
    while True:
        time.sleep(.1)
        if inst.query(':STAT:OPER?') == '256':
            break
    return True

inst.write(':INIT:CONT OFF')
freq = get_freq_from_ms2038c(inst)
data = {'freq':[], 'trace':[], 'vatt':[]}

p = bokeh.plotting.figure(height=300, toolbar_location=None, sizing_mode="stretch_width")
bokeh.plotting.output_notebook()
plot_spurious_multi(p, data)
for i, v in enumerate(range(0,0xD00,0x100)):
    set_vatt(port[8].upconv.vatt, v)
    sweep_ms2038c(inst)
    data['vatt'].append(hex(v))
    data['freq'].append(freq)
    data['trace'].append(get_trace_from_ms2038c(inst))
    clear_output()
    plot_spurious_multi(p, data)    

# 測定データの保存

In [ ]:
with open('if2000rf9600.pickle', 'wb') as f:
    pickle.dump(data, f)

# 測定データの復旧

In [ ]:
with open('if2000rf9600.pickle', 'rb') as f:
    data = pickle.load(f)